In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
import src.config as config

In [7]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28) #28

current_date=Timestamp('2023-10-22 07:00:00')


In [4]:
# current_date - timedelta(days=71)

In [5]:
# fetch_data_from - timedelta(days=7*52)

In [6]:
# prueba = fetch_data_from - timedelta(days=7*52)
# prueba.month

# from src.data import load_raw_data
# prueba = load_raw_data(year=2022, months=8)
# prueba

In [7]:
# fetch_data_from - timedelta(days=7*52)

In [8]:
# apoyo = fetch_data_from - timedelta(days=7*52)
# pruebe = prueba[prueba.pickup_datetime >=  apoyo]
# prueba

In [8]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # # download 2 files from website
    # rides = load_raw_data(year=from_date_.year) #, months=from_date_.month)
    # rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]
   
    # rides_2 = load_raw_data(year=to_date_.year) #, months=to_date_.month)
    # rides_2 = rides_2[(rides_2.pickup_datetime < to_date_) & (rides_2.pickup_datetime < to_date_)]

    # rides = pd.concat([rides, rides_2]) 

    # Intenta cargar los datos del primer año
    rides = load_raw_data(year=from_date_.year)
    rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]

    # Verifica si los años son diferentes
    if from_date_.year != to_date_.year:
    # Carga los datos del segundo año
        rides_2 = load_raw_data(year=to_date_.year)
        rides_2 = rides_2[(rides_2.pickup_datetime >= from_date_) & (rides_2.pickup_datetime < to_date_)]
        rides = pd.concat([rides, rides_2]) 


    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [10]:
# import subprocess
# from pathlib import Path

# def download_one_file_of_raw_data_with_wget(year: int) -> Path:
#     """
#     Downloads Parquet file with historical bike rides for the given `year`.
#     """
#     # URL del archivo a descargar
#     URL = f'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/bicicletas-publicas/recorridos-realizados-{year}.zip'

#     # Ruta de destino para guardar el archivo descargado
#     destination_path = RAW_DATA_DIR / f'recorridos-realizados-{year}.zip'

#     try:
#         # Utiliza wget para descargar el archivo en la ubicación deseada
#         subprocess.run(['wget', URL, '-O', destination_path])

#         # Verifica si el archivo se descargó correctamente
#         if destination_path.is_file():
#             print(f'Descargado año {year}')
#             return destination_path
#         else:
#             raise Exception(f'Error al descargar {URL}: El archivo no se descargó correctamente.')

#     except Exception as e:
#         raise Exception(f'Error al descargar {URL}: {str(e)}')

# # Uso de la función para descargar un archivo
# download_one_file_of_raw_data_with_wget(2022)

In [12]:
# from src.data import unzip_and_convert_csv_to_parquet
# unzip_and_convert_csv_to_parquet(2022)

In [9]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-09-24 07:00:00'), to_date_=Timestamp('2022-10-23 07:00:00')
Descargado año 2022


In [14]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 320/320 [00:01<00:00, 259.45it/s]


In [15]:
#Falta agregar lo de filtrar la fecha que corresponde, o quitar los meses que no queremos

In [16]:
rides.head()

,pickup_datetime,pickup_location_id,pickup_hour
1941202,2023-09-23 16:18:51,2,2023-09-23 16:00:00
1869025,2023-09-23 16:19:03,2,2023-09-23 16:00:00
2053076,2023-09-23 17:01:22,2,2023-09-23 17:00:00
2044124,2023-09-23 17:02:14,2,2023-09-23 17:00:00
2075829,2023-09-23 17:58:48,2,2023-09-23 17:00:00


In [17]:
ts_data.tail()

,pickup_hour,rides,pickup_location_id
215035,2023-10-21 11:00:00,0,443
215036,2023-10-21 12:00:00,0,443
215037,2023-10-21 13:00:00,0,443
215038,2023-10-21 14:00:00,0,443
215039,2023-10-21 15:00:00,0,443


In [18]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/100501
Connected. Call `.close()` to terminate connection gracefully.


In [19]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/215040 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/100501/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x25e10bf9cd0>, None)